# Post-Training Quantization in Keras using the Model Compression Toolkit (MCT)
[Run this tutorial in Google Colab](https://colab.research.google.com/github/sony/model_optimization/blob/main/tutorials/notebooks/mct_features_notebooks/keras/example_keras_post-training_quantization.ipynb)

## Overview
This quick-start guide explains how to use the **Model Compression Toolkit (MCT)** to quantize a Keras model. We will load a pre-trained model and  quantize it using the MCT with **Post-Training Quatntization (PTQ)**. Finally, we will evaluate the quantized model and export it to a Keras or TFLite files.

## Summary
In this tutorial, we will cover:

1. Loading and preprocessing ImageNet’s validation dataset.
2. Constructing an unlabeled representative dataset.
3. Post-Training Quantization using MCT.
4. Accuracy evaluation of the floating-point and the quantized models.
5. Exporting the model to Keras and TFLite files.

## Setup
Install the relevant packages:

In [ ]:
TF_VER = '2.14'
!pip install -q tensorflow[and-cuda]~={TF_VER}

In [ ]:
import importlib
if not importlib.util.find_spec('model_compression_toolkit'):
    !pip install model_compression_toolkit

In [ ]:
import keras
import tensorflow as tf

Load a pre-trained MobileNetV2 model from Keras, in 32-bits floating-point precision format.

In [ ]:
from keras.applications.mobilenet_v2 import MobileNetV2

float_model = MobileNetV2()

## Dataset preparation
### Download the ImageNet validation set
Download the ImageNet dataset with only the validation split.
**Note:** For demonstration purposes we use the validation set for the model quantization routines. Usually, a subset of the training dataset is used, but loading it is a heavy procedure that is unnecessary for the sake of this demonstration.

This step may take several minutes...

In [ ]:
import os
 
if not os.path.isdir('imagenet'):
    !mkdir imagenet
    !wget -P imagenet https://image-net.org/data/ILSVRC/2012/ILSVRC2012_devkit_t12.tar.gz
    !wget -P imagenet https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar
    
    !cd imagenet && tar -xzf ILSVRC2012_devkit_t12.tar.gz && \
     mkdir ILSVRC2012_img_val && tar -xf ILSVRC2012_img_val.tar -C ILSVRC2012_img_val

The following code organizes the extracted data into separate folders for each label, making it compatible with Keras dataset loaders.

In [ ]:
from pathlib import Path
import shutil

root = Path('./imagenet')
imgs_dir = root / 'ILSVRC2012_img_val'
target_dir = root /'val'

def extract_labels():
    !pip install -q scipy
    import scipy
    mat = scipy.io.loadmat(root / 'ILSVRC2012_devkit_t12/data/meta.mat', squeeze_me=True)
    cls_to_nid = {s[0]: s[1] for i, s in enumerate(mat['synsets']) if s[4] == 0} 
    with open(root / 'ILSVRC2012_devkit_t12/data/ILSVRC2012_validation_ground_truth.txt', 'r') as f:
        return [cls_to_nid[int(cls)] for cls in f.readlines()]

if not target_dir.exists():
    labels = extract_labels()
    for lbl in set(labels):
        os.makedirs(target_dir / lbl)
    
    for img_file, lbl in zip(sorted(os.listdir(imgs_dir)), labels):
        shutil.move(imgs_dir / img_file, target_dir / lbl)


These functions generate a `tf.data.Dataset` from image files in a directory.

In [ ]:
def imagenet_preprocess_input(images, labels):
    return tf.keras.applications.mobilenet_v2.preprocess_input(images), labels

def get_dataset(batch_size, shuffle):
    dataset = tf.keras.utils.image_dataset_from_directory(
        directory='./imagenet/val',
        batch_size=batch_size,
        image_size=[224, 224],
        shuffle=shuffle,
        crop_to_aspect_ratio=True,
        interpolation='bilinear')
    dataset = dataset.map(lambda x, y: (imagenet_preprocess_input(x, y)), num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

## Representative Dataset
For quantization with MCT, we need to define a representative dataset required by the PTQ algorithm. This dataset is a generator that returns a list of images:

In [ ]:
batch_size = 32
n_iter = 10

dataset = get_dataset(batch_size, shuffle=True)

def representative_dataset_gen():
    for _ in range(n_iter):
        yield [dataset.take(1).get_single_element()[0].numpy()]

## Target Platform Capabilities
MCT optimizes the model for dedicated hardware. This is done using TPC (for more details, please visit our [documentation](https://sony.github.io/model_optimization/docs/api/api_docs/modules/target_platform.html)). Here, we use the default Tensorflow TPC:

In [ ]:
import model_compression_toolkit as mct

# Get a TargetPlatformCapabilities object that models the hardware for the quantized model inference. Here, for example, we use the default platform that is attached to a Keras layers representation.
target_platform_cap = mct.get_target_platform_capabilities('tensorflow', 'default')

## Post-Training Quantization using MCT
Now for the exciting part! Let’s run PTQ on the model.

In [ ]:
quantized_model, quantization_info = mct.ptq.keras_post_training_quantization(
        in_model=float_model,
        representative_data_gen=representative_dataset_gen,
        target_platform_capabilities=target_platform_cap
)

Our model is now quantized. MCT has created a simulated quantized model within the original Keras framework by inserting [quantization representation modules](https://github.com/sony/mct_quantizers). These modules, such as `KerasQuantizationWrapper` and `KerasActivationQuantizationHolder`, wrap Keras layers to simulate the quantization of weights and activations, respectively. While the size of the saved model remains unchanged, all the quantization parameters are stored within these modules and are ready for deployment on the target hardware. In this example, we used the default MCT settings, which compressed the model from 32 bits to 8 bits, resulting in a compression ratio of 4x.

## Model Evaluation
In order to evaluate our models, we first need to load the validation dataset.

In [ ]:
val_dataset = get_dataset(batch_size=50, shuffle=False)

Let's start with the floating-point model evaluation. We need to compile the model before evaluation and set the loss and the evaluation metric.

In [ ]:
float_model.compile(loss=keras.losses.SparseCategoricalCrossentropy(), metrics="accuracy")
float_accuracy = float_model.evaluate(val_dataset)
print(f"Float model's Top 1 accuracy on the Imagenet validation set: {(float_accuracy[1] * 100):.2f}%")

Finally, let's evaluate the quantized model:

In [ ]:
quantized_model.compile(loss=keras.losses.SparseCategoricalCrossentropy(), metrics="accuracy")
quantized_accuracy = quantized_model.evaluate(val_dataset)
print(f"Quantized model's Top 1 accuracy on the Imagenet validation set: {(quantized_accuracy[1] * 100):.2f}%")

You can see that we got a very small degradation with a compression rate of x4 !
Now, we can export the quantized model to Keras and TFLite:

In [ ]:
mct.exporter.keras_export_model(
    model=quantized_model,
    save_model_path='qmodel.tflite',
    serialization_format=mct.exporter.KerasExportSerializationFormat.TFLITE,
    quantization_format=mct.exporter.QuantizationFormat.FAKELY_QUANT)

mct.exporter.keras_export_model(model=quantized_model, save_model_path='qmodel.keras')

## Conclusion

In this tutorial, we demonstrated how to quantize a classification model in a hardware-friendly manner using MCT. We observed that a 4x compression ratio was achieved with minimal performance degradation.

The key advantage of hardware-friendly quantization is that the model can run more efficiently in terms of runtime, power consumption, and memory usage on designated hardware.

MCT can deliver competitive results across a wide range of tasks and network architectures. For more details, [check out the paper](https://arxiv.org/abs/2109.09113).

## Copyrights

Copyright 2024 Sony Semiconductor Israel, Inc. All rights reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
